In [1]:
!pip3 install Cython
!pip3 install pomegranate
#Nous allons commencer par importer du module Pomegranate les éléments dont nous aurons besoin :
from pomegranate import *
from pomegranate import HiddenMarkovModel as Model

In [2]:
#Import d'outils pythons
import random
import operator
import math
import pandas as pd
import numpy 

Code récupéré du tp1 pour récupérer les séries des rendements journaliers discrétisés en
{−1, 0, +1}

In [3]:
#Import dataset  de la Bourse de Paris
dataParis =  pd.read_csv("^FCHI.csv")
#Import dataset de la Bourse de Francfort
dataFrancfort = pd.read_csv("^GDAXI.csv")
#Import dataset  de la Bourse de NewYork
dataNY =  pd.read_csv("^DJI.csv")
#Import dataset  de la Bourse de societe Hi-tech
dataHiTech =  pd.read_csv("^IXIC.csv")
##On voit que les 2 dataset n'ont pas le même nombre de lignes donc pour les prochaines calcules ont va fusionner nos dataframe avec la date et remplir les trous
adjNYDate = dataNY[["Date","Adj Close"]]
adjParisDate = dataParis[["Date","Adj Close"]]
adjFCDate = dataFrancfort[["Date","Adj Close"]]
adjHiTechDate = dataHiTech[["Date","Adj Close"]]

# ON renomme les colonnes pour savoir a qui correspond les données
adjNYDate = adjNYDate.rename(columns={"Adj Close": "NY Adj Close"})
adjParisDate = adjParisDate.rename(columns={"Adj Close": "Paris Adj Close"})
adjFCDate = adjFCDate.rename(columns={"Adj Close": "FC Adj Close"})
adjHiTechDate = adjHiTechDate.rename(columns={"Adj Close": "HT Adj Close"})

adjParisNY = pd.merge(adjParisDate, adjNYDate, how='outer')
adjHTFC = pd.merge(adjHiTechDate,adjFCDate, how='outer')
adjAll = pd.merge(adjHTFC,adjParisNY, how='outer')

##On remplace les NaN avec les valeurs au alentour
def isNan(data, column):
    for i in range(len(data)-1):
        if math.isnan(data[column].loc[i]) :
            if i == 0 :
                data[column].loc[i] = data[column].loc[i+4]
            elif i < len(data):
                data[column].loc[i] = data[column].loc[i+1]
            else : 
                data[column].loc[i] = data[column].loc[i-1]
    return data
##On applique la fct
adjAll = isNan(adjAll,"Paris Adj Close")
adjAll = isNan(adjAll,"NY Adj Close")
adjAll = isNan(adjAll,"FC Adj Close")
adjAll = isNan(adjAll,"HT Adj Close")

##On fait des dataset contenant que la colonne qui nous interesse
adjParis = adjAll["Paris Adj Close"]
adjNY = adjAll["NY Adj Close"]
adjFC = adjAll["FC Adj Close"]
adjHT = adjAll["HT Adj Close"]

# +1 signifie que la valeur de l’indice a augmenté, −1 qu’elle a diminué et 0 qu’il
# n’y a pas eu de mouvement digne de note
def createSerie(adj):
    serie = []

    for i in range(len(adj)):
        if i == 0 :
            serie.append("0")
        else :
            ln = math.log(adj[i]/adj[i-1])
            if ln > 0.005 :
                serie.append("1")
            elif ln <= 0.005 and ln >= -0.005 :
                serie.append("0")
            else :
                serie.append("-1")
    return serie

serieParis = createSerie(adjParis)
serieNY = createSerie(adjNY)
serieFC = createSerie(adjFC)
serieHT = createSerie(adjHT)

/home/deptinfo/.local/lib/python3.8/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


"Le modèle qu’on construira ne pourrait être plus simple : on fera l’hypothèse qu’il n’existent que deux tendances : à la hausse (bullish, du taureau, dans le jargon des traders) et à la baisse (bearish, de l’ours). Ces deux tendances sont les deux états “cachés” (non observables directement) de notre MMC ; les variations quotidiennes sont les observations “émises” par chaque tendance, suivant sa propre distribution de probabilité."

1. Construisez avec Pomegranate le MMC décrit ci-dessus. Pour commencer, vous mettrez des probabilités arbitraires. Attention : pour quelque raison mistérieuse, Pomegranate ajoute toujours un état initial start et un état final end même si vous dites expressément au constructeur de la classe HiddenMarkovModel que vous ne les voulez pas. Il suffit alors d’ajouter une transition de probabilité 1/2 de start à bullish ainsi que de start à bearish.

In [4]:
bullish = DiscreteDistribution({'-1' : 0.25, '0' : 0.40,  '1' : 0.35})
bearish = DiscreteDistribution({'-1' : 0.30, '0' : 0.55, '1' : 0.15 })

In [5]:
s1 = State( bullish , name="bullish" )
s2 = State( bearish, name="bearish" )

In [6]:
model = HiddenMarkovModel('bourse')
model.add_states([s1, s2])
model.add_transition( model.start, s1, 0.50 )
model.add_transition( model.start, s2, 0.50 )
model.add_transition( s1, s1, 0.80 )
model.add_transition( s1, s2, 0.20 )
model.add_transition( s2, s2, 0.90 )
model.add_transition( s2, s1, 0.10 )
model.add_transition( s1, model.end, 0.50 )
model.add_transition( s2, model.end, 0.50 )
model.bake()

2. Créez, avec un petit script Python, une série historique de test, où vous allez simuler un marché qui alterne, sur un total di 1000 jours, 50 jours de taureau et 50 jours d’ours ; lorsque le marché est bullish,

        P(st = −1) = 0.2, P(st = 0) = 0.3, P(st = +1) = 0.5,

        et lorsqu’il est bearish,

        P(st = −1) = 0.5, P(st = 0) = 0.2, P(st = +1) = 0.3.

In [7]:
etat = []
marche = []
bu = ['bullish']*50
be = ['bearish']*50
d = bu + be
for i in range(10) :
    etat = etat + d

for j in range(len(etat)):
    if etat[j] == "bullish":
        #P(st = −1) = 0.2, P(st = 0) = 0.3, P(st = +1) = 0.5,  
        p = ['-1','-1','0','0','0','1','1','1','1','1']
        marche.append(random.choice(p)) 
    elif etat[j] =="bearish" :
        #P(st = −1) = 0.5, P(st = 0) = 0.2, P(st = +1) = 0.3.
        p = ['-1','-1','-1','-1','-1','0','0','1','1']
        marche.append(random.choice(p))    

3. Entraînez le MMC (méthode fit) avec cette série de test. Une fois entraîné, le MMC devrait reconnaître (méthode viterbi) les deux tendances, au fur et à mesure qu’elles s’alternent. Vérifiez aussi que les probabilités d’émission des deux états reflètent bien les paramètres que vous avez utilisé pour produire la série de test.

In [8]:
#La modélisation semble très bien fonctionner avec la serie teste car on retrouve les probabilités généré dans la serie soit ;
# Bullish : P(st = −1) = 0.2, P(st = 0) = 0.3, P(st = +1) = 0.5,
# Bearish : P(st = −1) = 0.5, P(st = 0) = 0.2, P(st = +1) = 0.3.
sequence = list(numpy.asarray(marche))
model.fit([sequence], algorithm='viterbi')
model.viterbi(marche)
print("bullish : " , bullish.parameters)
print("bearish : " ,bearish.parameters)

bullish :  [{'-1': 0.20297951582867785, '0': 0.2905027932960894, '1': 0.5065176908752328}]
bearish :  [{'-1': 0.5809935205183585, '0': 0.21166306695464362, '1': 0.20734341252699784}]


In [15]:
#print(", ".join( state.name for i, state in model.viterbi(marche)[1] ))

4. Maintenant, appliquez le MMC aux séries réelles (celles que vous avez obtenues lors du TD n◦ 1). Cela veut dire entraîner le MMC avec une série, puis lui faire reconnaître les tendances sur la même série.

In [18]:
##Fct qui va permettre d'appliquer le modèle aux series. 
def mmcseries(data) :
    bullish = DiscreteDistribution({'-1' : 0.25, '0' : 0.40,  '1' : 0.35})
    bearish = DiscreteDistribution({'-1' : 0.30, '0' : 0.55, '1' : 0.15 })
    s1 = State( bullish , name="bullish" )
    s2 = State( bearish, name="bearish" )
    model = HiddenMarkovModel('bourse')
    model.add_states([s1, s2])
    model.add_transition( model.start, s1, 0.50 )
    model.add_transition( model.start, s2, 0.50 )
    model.add_transition( s1, s1, 0.80 )
    model.add_transition( s1, s2, 0.20 )
    model.add_transition( s2, s2, 0.90 )
    model.add_transition( s2, s1, 0.10 )
    model.add_transition( s1, model.end, 0.50 )
    model.add_transition( s2, model.end, 0.50 )
    model.bake()
    sequence = list(numpy.asarray(data))
    model.fit([sequence], algorithm='viterbi')
    model.viterbi(data)
    print("bullish : " , bullish.parameters)
    print("bearish : " ,bearish.parameters)
    print(", ".join( state.name for i, state in model.viterbi(data)[1] ))

In [19]:
mmcseries(serieHT) 

bullish :  [{'-1': 0.29936305732484075, '0': 0.16560509554140126, '1': 0.535031847133758}]
bearish :  [{'-1': 0.28846153846153844, '0': 0.4423076923076923, '1': 0.2692307692307692}]
bourse-start, bearish, bearish, bearish, bearish, bearish, bearish, bearish, bearish, bearish, bearish, bearish, bearish, bearish, bearish, bearish, bearish, bearish, bearish, bearish, bearish, bearish, bearish, bearish, bearish, bearish, bearish, bearish, bearish, bearish, bearish, bearish, bearish, bearish, bearish, bearish, bearish, bearish, bearish, bearish, bearish, bearish, bearish, bearish, bearish, bearish, bearish, bearish, bearish, bearish, bearish, bearish, bearish, bearish, bearish, bearish, bearish, bearish, bearish, bearish, bearish, bearish, bearish, bearish, bearish, bearish, bearish, bearish, bearish, bearish, bearish, bearish, bearish, bearish, bullish, bullish, bullish, bullish, bullish, bullish, bullish, bullish, bullish, bullish, bullish, bullish, bullish, bullish, bullish, bullish, bul

<img src="^IXIC.png">

Le modèle n'a pas l'air de très bien fonctionner ici

In [20]:
mmcseries(serieFC) 

bullish :  [{'-1': 0.3153153153153153, '0': 0.1891891891891892, '1': 0.4954954954954955}]
bearish :  [{'-1': 0.2733333333333333, '0': 0.4666666666666667, '1': 0.26}]
bourse-start, bearish, bearish, bearish, bearish, bearish, bearish, bearish, bearish, bearish, bearish, bearish, bearish, bearish, bearish, bearish, bearish, bearish, bearish, bearish, bearish, bearish, bearish, bearish, bearish, bearish, bearish, bearish, bearish, bearish, bearish, bearish, bearish, bearish, bearish, bearish, bearish, bearish, bearish, bearish, bearish, bearish, bearish, bearish, bearish, bearish, bearish, bearish, bearish, bearish, bearish, bearish, bearish, bearish, bearish, bearish, bearish, bearish, bearish, bearish, bearish, bearish, bearish, bearish, bearish, bearish, bearish, bearish, bearish, bearish, bearish, bearish, bearish, bearish, bearish, bullish, bullish, bullish, bullish, bullish, bullish, bullish, bullish, bullish, bullish, bullish, bullish, bullish, bullish, bullish, bullish, bullish, b

<img src="^GDAXI.png">

Le modèle a l'air d'ete fidèle a la courbe mais avec un certaine décalage

In [21]:
mmcseries(serieParis)

bullish :  [{'-1': 0.33640552995391704, '0': 0.2903225806451613, '1': 0.37327188940092165}]
bearish :  [{'-1': 0.11363636363636363, '0': 0.7045454545454546, '1': 0.18181818181818182}]
bourse-start, bearish, bearish, bearish, bearish, bearish, bearish, bearish, bearish, bearish, bearish, bearish, bearish, bearish, bearish, bearish, bearish, bearish, bearish, bearish, bearish, bearish, bearish, bearish, bearish, bearish, bearish, bearish, bearish, bearish, bearish, bearish, bearish, bearish, bearish, bearish, bearish, bearish, bearish, bearish, bearish, bearish, bearish, bearish, bearish, bullish, bullish, bullish, bullish, bullish, bullish, bullish, bullish, bullish, bullish, bullish, bullish, bullish, bullish, bullish, bullish, bullish, bullish, bullish, bullish, bullish, bullish, bullish, bullish, bullish, bullish, bullish, bullish, bullish, bullish, bullish, bullish, bullish, bullish, bullish, bullish, bullish, bullish, bullish, bullish, bullish, bullish, bullish, bullish, bullish, b

<img src="^FCHI.png">

Le modèle fonctionne très bien avec cette serie.

In [22]:
mmcseries(serieNY)

bullish :  [{'-1': 0.3443396226415094, '0': 0.25943396226415094, '1': 0.39622641509433965}]
bearish :  [{'-1': 0.12244897959183673, '0': 0.6938775510204082, '1': 0.1836734693877551}]
bourse-start, bearish, bearish, bearish, bearish, bearish, bearish, bearish, bearish, bearish, bearish, bearish, bearish, bearish, bearish, bearish, bearish, bearish, bearish, bearish, bearish, bearish, bearish, bearish, bearish, bearish, bearish, bearish, bearish, bearish, bearish, bearish, bearish, bearish, bearish, bearish, bearish, bearish, bearish, bearish, bearish, bearish, bearish, bearish, bearish, bearish, bearish, bearish, bearish, bullish, bullish, bullish, bullish, bullish, bullish, bullish, bullish, bullish, bullish, bullish, bullish, bullish, bullish, bullish, bullish, bullish, bullish, bullish, bullish, bullish, bullish, bullish, bullish, bullish, bullish, bullish, bullish, bullish, bullish, bullish, bullish, bullish, bullish, bullish, bullish, bullish, bullish, bullish, bullish, bullish, bu

<img src="^DJI.png">

Le model a l'air de plutot bien fonctionner
Ce n'est pas super précis, mais on a la tendance (bullish ou bearish) qui fonctionne
C'est un peu décalé mais ca reste fidèle.